In [1]:
# install neptune
install.packages('neptune', dependencies = TRUE)

# install other packages for this tutorial
install.packages(c('caret','digest', 'mlbench', 'randomForest'), dependencies = TRUE)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rappdirs’, ‘reticulate’


Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘numDeriv’, ‘SQUAREM’, ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘fastmap’, ‘gdata’, ‘caTools’, ‘TH.data’, ‘profileModel’, ‘minqa’, ‘nloptr’, ‘statmod’, ‘RcppEigen’, ‘plotrix’, ‘lava’, ‘shiny’, ‘miniUI’, ‘classInt’, ‘labelled’, ‘gplots’, ‘libcoin’, ‘matrixStats’, ‘multcomp’, ‘iterators’, ‘data.table’, ‘gower’, ‘timeDate’, ‘brglm’, ‘gtools’, ‘lme4’, ‘qvcalc’, ‘Formula’, ‘plotmo’, ‘TeachingDemos’, ‘prodlim’, ‘combinat’, ‘questionr’, ‘ROCR’, ‘mvtnorm’, ‘modeltools’, ‘strucchange’, ‘coin’, ‘zoo’, ‘sandwich’, ‘ISwR’, ‘corpcor’, ‘foreach’, ‘ModelMetrics’, ‘recipes’, ‘pROC’, ‘BradleyTerry2’, ‘e1071’, ‘earth’, ‘fastICA’, ‘gam’, ‘ipred’, ‘kernlab’, ‘klaR’, ‘ellipse’, ‘mda’, ‘MLmetrics’, ‘party’, ‘pls’, ‘proxy’, ‘RANN’, ‘spls’, ‘subselec

In [0]:
# load libraries
library(neptune)
library(caret)
library(digest)
library(mlbench)
library(randomForest)

SEED=1234
set.seed(SEED)

# load dataset
data(Sonar)
dataset <- Sonar
x <- dataset[,1:60]   # predictors
y <- dataset[,61]     # labels

In [42]:
# Initialize Neptune
# You may want to specify your python environment with 'python' and 'python_path' arguments.
init_neptune(project_name = 'shared/r-integration',
             api_token = 'ANONYMOUS')

couldn't import neptune client. Trying to install



ERROR: ignored

In [0]:
# Start an experiment and track hyperparameters

params = list(metric="Accuracy",
              tuneLength=100,
              model="rf",
              searchMethod="random",
              cvMethod="repeatedcv",
              cvFolds=2,
              cvRepeats=1)

create_experiment(name='training on Sonar',
                  tags=c('random-forest','sonar', 'caret'),
                  params = params
)

In [0]:
# track data version and SEED
set_property(property = 'data-version', value = digest(dataset))
set_property(property = 'seed', value = SEED)

In [0]:
# train your model
control <- trainControl(method=params$cvMethod,
                        number=params$cvFolds,
                        repeats=params$cvRepeats,
                        search=params$searchMethod)

model <- train(Class ~ ., data=dataset,
                method=params$model,
                metric=params$metric,
                tuneLength=params$tuneLength,
                trControl=control)

In [0]:
# Log metrics
scores <- getTrainPerf(model)
log_metric('Train Accuracy', scores$TrainAccuracy)
log_metric('Train Kappa', scores$TrainKappa)

for (name in names(model$bestTune)) {
  set_property(name, model$bestTune[[name]])
}

for (acc in model$results$Accuracy){
  log_metric('Accuracy', acc)
}

for (kappa in model$results$Kappa){
  log_metric('Kappa', kappa)
}

In [0]:
# Log artifact
save(model, file="model.Rdata")
log_artifact('model.Rdata')

In [0]:
# Log image
ggplot(model) + geom_line(color='steelblue') + geom_point(color='steelblue', size=5)
ggsave('param_plot.jpeg')
log_image('parameter_search', 'param_plot.jpeg')

In [0]:
# Stop Experiment
stop_experiment()